# EDA - Prepare Data for Model

## Load Librabries

In [1]:
import pandas as pd
import Functions_Books as funct
import nltk as nltk

from nltk.corpus import stopwords #Natural Language Toolkit
from nltk.tokenize import RegexpTokenizer
#nltk.download('RegexpTokenizer')

#import spacy.cli
#spacy.cli.download("fr_core_news_sm")
#FrenchNLP = spacy.load("fr_core_news_sm")

## Read cleaned data into dataframe

In [2]:
data_cleaned=pd.read_csv('DataFinal\DataCleanedFinal.csv',index_col=0)
data_preproc=data_cleaned.copy()

In [3]:
data_preproc.head()

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating
0,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,1439550417,"Classics, Fiction, Historical, Historical Fict...",The unforgettable novel of a childhood in a sl...,324,1960,English,https://i.gr-assets.com/images/S/compressed.ph...,5011381,4.27
1,Harry Potter and the Sorcerer's Stone,Harry Potter #1,J.K. Rowling,0,"Fantasy, Fiction, Young Adult, Magic, Children...",Harry Potter's life is miserable. His parents ...,309,1997,English,https://i.gr-assets.com/images/S/compressed.ph...,8003482,4.47
2,The Diary of a Young Girl,none,"Anne Frank, Eleanor Roosevelt, Barbara Mooyaar...",330107372,"Nonfiction, Classics, History, Biography, Auto...",Discovered in the attic in which she spent the...,283,1947,English,https://i.gr-assets.com/images/S/compressed.ph...,3088199,4.16
3,Animal Farm,none,"George Orwell, Russell Baker, C.M. Woodhouse",9780451526342,"Classics, Fiction, Science Fiction, Dystopia, ...",Librarian's note: There is an Alternate Cover ...,141,1945,English,https://i.gr-assets.com/images/S/compressed.ph...,3095756,3.97
4,The Little Prince,none,"Antoine de Saint-Exupéry, Richard Howard",3125971403,"Classics, Fiction, Fantasy, Childrens, Cultura...",A pilot stranded in the desert awakes one morn...,96,1943,English,https://i.gr-assets.com/images/S/compressed.ph...,1609561,4.31


In [4]:
data_preproc.shape

(8431, 12)

## Text Preprocessing - Tokenizing with NLTK

Info found here: https://towardsdatascience.com/5-simple-ways-to-tokenize-text-in-python-92c6804edfc4


### Add column Describtion_tokenized and Title_tokenized

In [5]:
selected_columns = data_preproc[["title","description","series"]]
newdf=selected_columns.copy()
newdf = newdf.rename(columns={'title': 'title_tokenized', 'description': 'desccription_tokenized', 'series':'series_tokenized'})
frames=[data_preproc,newdf]
DataForPreprocessing = pd.concat(frames,axis=1)
DataForPreprocessing.head()

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
0,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,1439550417,"Classics, Fiction, Historical, Historical Fict...",The unforgettable novel of a childhood in a sl...,324,1960,English,https://i.gr-assets.com/images/S/compressed.ph...,5011381,4.27,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,To Kill a Mockingbird
1,Harry Potter and the Sorcerer's Stone,Harry Potter #1,J.K. Rowling,0,"Fantasy, Fiction, Young Adult, Magic, Children...",Harry Potter's life is miserable. His parents ...,309,1997,English,https://i.gr-assets.com/images/S/compressed.ph...,8003482,4.47,Harry Potter and the Sorcerer's Stone,Harry Potter's life is miserable. His parents ...,Harry Potter #1
2,The Diary of a Young Girl,none,"Anne Frank, Eleanor Roosevelt, Barbara Mooyaar...",330107372,"Nonfiction, Classics, History, Biography, Auto...",Discovered in the attic in which she spent the...,283,1947,English,https://i.gr-assets.com/images/S/compressed.ph...,3088199,4.16,The Diary of a Young Girl,Discovered in the attic in which she spent the...,none
3,Animal Farm,none,"George Orwell, Russell Baker, C.M. Woodhouse",9780451526342,"Classics, Fiction, Science Fiction, Dystopia, ...",Librarian's note: There is an Alternate Cover ...,141,1945,English,https://i.gr-assets.com/images/S/compressed.ph...,3095756,3.97,Animal Farm,Librarian's note: There is an Alternate Cover ...,none
4,The Little Prince,none,"Antoine de Saint-Exupéry, Richard Howard",3125971403,"Classics, Fiction, Fantasy, Childrens, Cultura...",A pilot stranded in the desert awakes one morn...,96,1943,English,https://i.gr-assets.com/images/S/compressed.ph...,1609561,4.31,The Little Prince,A pilot stranded in the desert awakes one morn...,none


### Explore Book Languages

We have books in different languages. Due to time reasons we will only process the English books with NLTK.
All other languages could be processed with spaCy at a later point in time.<br>


In [6]:
DataForPreprocessing['language'].value_counts()

English               7606
none                   542
Arabic                  55
French                  37
German                  31
Spanish                 29
Persian                 17
Portuguese              16
Dutch                   14
Polish                  12
Italian                  7
Romanian                 7
Turkish                  7
Indonesian               6
Multiple languages       5
Swedish                  4
Greek                    4
Tamil                    3
Malayalam                2
Finnish                  2
Hindi                    2
Filipino, Pilipino       2
Ukrainian                2
Russian                  2
Urdu                     2
Norwegian                1
Telugu                   1
Catalan, Valencian       1
Hungarian                1
Tagalog                  1
Serbian                  1
Bulgarian                1
Scots                    1
Kannada                  1
Japanese                 1
Danish                   1
Croatian                 1
M

###  Deal with different languages by preprocessing title

#### Explore books with language 'None'

In [7]:
NoneLangBooks=pd.DataFrame(DataForPreprocessing[DataForPreprocessing['language']=='none'])
NoneLangBooks.shape

(542, 15)

In [8]:
#Return lower cases
funct.return_Serieslowercase(NoneLangBooks,'title_tokenized')

In [9]:
#Remove stop words
NoneLangBooks['title_tokenized'] = NoneLangBooks.title_tokenized.apply(func=funct.remove_stopwords)

In [10]:
#Remove non-Ascii Characters
NoneLangBooks['title_tokenized'] = NoneLangBooks['title_tokenized'].apply(func=funct.remove_nonAscii)

In [11]:
#Remove punctuation
NoneLangBooks['title_tokenized'] = NoneLangBooks['title_tokenized'].apply(func=funct.remove_punctuation)
NoneLangBooks[NoneLangBooks['title_tokenized']=='']

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
987,كلاب الراعي,none,أشرف العشماوي,0,"Novels, Historical, Historical Fiction",.. هذه الرواية تأسرك منذ فصولها الأولى ، وقد أ...,355,2015,none,https://i.gr-assets.com/images/S/compressed.ph...,7279,4.16,,.. هذه الرواية تأسرك منذ فصولها الأولى ، وقد أ...,none
1158,سيدة الزمالك,none,أشرف العشماوي,9777951620,"Novels, Fiction, Historical, Historical Fictio...","ليلة الحادث تسلل خمسة أشخاص لفيلا ""سولومون شيك...",375,2018,none,https://i.gr-assets.com/images/S/compressed.ph...,12414,4.25,,"ليلة الحادث تسلل خمسة أشخاص لفيلا ""سولومون شيك...",none
3633,سرقات مشروعة,none,أشرف العشماوي,0,"History, Nonfiction, Northern Africa, Egypt",يستعرض هذا الكتاب صفحات مجهولة من تاريخ سرقة و...,312,2011,none,https://i.gr-assets.com/images/S/compressed.ph...,988,4.26,,يستعرض هذا الكتاب صفحات مجهولة من تاريخ سرقة و...,none
5103,سال بلوا,none,عباس معروفی,9463113744,"Novels, Fiction, Cultural, Iran, Literature, R...",می‌دانی اولین بوسهٔ جهان چه‌طور کشف شد؟دست‌هاش...,342,1992,none,https://i.gr-assets.com/images/S/compressed.ph...,5515,3.94,,می‌دانی اولین بوسهٔ جهان چه‌طور کشف شد؟دست‌هاش...,none
5157,تراژدی اسبی که ایستاد,none,روح اله سهرابی,0,none,در بخشی از این رمان می‌خوانیم:با این درخواست گ...,200,2013,none,https://i.gr-assets.com/images/S/compressed.ph...,280,1.91,,در بخشی از این رمان می‌خوانیم:با این درخواست گ...,none
5970,All of Me,none,Kevine Walcott,9781095642580,"Contemporary, Novels, Fiction, Epic, Drama, Su...","Kevine Walcott's 'All of Me' is a holistic, bo...",38,2019,none,NaN,761,4.42,,"Kevine Walcott's 'All of Me' is a holistic, bo...",none
7131,मधुशाला,none,"Harivansh Rai Bachchan, हरिवंश राय बच्चन",9780140120097,"Poetry, Classics, Cultural, India, Asian Liter...",Dr Harivanshrai Bachchan is India's leading Hi...,152,1935,none,https://i.gr-assets.com/images/S/compressed.ph...,3288,4.45,,Dr Harivanshrai Bachchan is India's leading Hi...,none
8393,أرض الغجر,none,أحمد أبو هيبة,0,none,None,239,2014,none,https://i.gr-assets.com/images/S/compressed.ph...,180,3.86,,None,none


 #### Remove non-English books

The result above shows all non-English books after tokenizing.

In [12]:
NoneLangBooks=NoneLangBooks[NoneLangBooks['title_tokenized']!='']
NoneLangBooks

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
283,Mistress Suffragette,none,Diana Forbes,1946409073,"Fiction, Novels, Drama, Contemporary, Historic...",A young woman without prospects at a ball in G...,333,2017,none,https://i.gr-assets.com/images/S/compressed.ph...,12973,4.32,mistress suffragette,A young woman without prospects at a ball in G...,none
350,Stones (One True Child #5),none,L.C. Conn,0,"Book Club, Novels, Fiction, Drama, Contemporary","Deep within the New Zealand bush, lies a plate...",327,2019,none,https://i.gr-assets.com/images/S/compressed.ph...,569,4.31,stones one true child 5,"Deep within the New Zealand bush, lies a plate...",none
355,Mission: Subhero,none,Linda Armstrong,0,"Contemporary, Novels, Fiction, Drama, Book Clu...","With just a few lines of code, one average man...",225,2017,none,https://i.gr-assets.com/images/S/compressed.ph...,2417,4.35,mission subhero,"With just a few lines of code, one average man...",none
356,A Summer of Good-Byes (Blue Triangle Press Boo...,none,Fred Misurella,0,"Fiction, Novels, Adult, Contemporary, Adventur...",PASSION IN PROVENCE:Ben and Lee Alto follow Va...,0,2017,none,https://i.gr-assets.com/images/S/compressed.ph...,1713,4.36,summer good byes blue triangle press book 2,PASSION IN PROVENCE:Ben and Lee Alto follow Va...,none
357,The Scopas Factor,none,Vincent Panettiere,9781543947717,"Fiction, Contemporary, Novels, Drama, Book Clu...","A Hmong ""story cloth,"" a Revolutionary War bat...",326,2018,none,https://i.gr-assets.com/images/S/compressed.ph...,1080,4.31,scopas factor,"A Hmong ""story cloth,"" a Revolutionary War bat...",none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8388,Aj's Ireland: A Christmas Comedy,A.J. #0.5,Renee Riva,9780692329047,"Humor, Family, Adventure","In 1967, nine-year-old AJ wins a trip to Irela...",174,2014,none,https://i.gr-assets.com/images/S/compressed.ph...,23,4.39,aj s ireland christmas comedy,"In 1967, nine-year-old AJ wins a trip to Irela...",A.J. #0.5
8389,Shift!,none,Bill C. Castengera,9781500887032,none,Humankind is evolving. Technology has advanced...,336,2014,none,https://i.gr-assets.com/images/S/compressed.ph...,5,3.80,shift,Humankind is evolving. Technology has advanced...,none
8397,"Talent for Humanity: Stories of Creativity, Co...",none,Patrick Gaffney,9781626341814,"Self Help, Nonfiction",We are all born with the power to imagine what...,208,2015,none,https://i.gr-assets.com/images/S/compressed.ph...,24,4.38,talent humanity stories creativity compassion ...,We are all born with the power to imagine what...,none
8399,Ola: The Tale of a Young Moon Maiden,none,D.C. Nkado,9781312035,none,"Set in the distant land of Odu, the witches of...",0,2014,none,https://i.gr-assets.com/images/S/compressed.ph...,2,4.50,ola tale young moon maiden,"Set in the distant land of Odu, the witches of...",none


#### Set language to English for English books

In [13]:
NoneLangBooks['language']='English'
NoneLangBooks.head()

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
283,Mistress Suffragette,none,Diana Forbes,1946409073,"Fiction, Novels, Drama, Contemporary, Historic...",A young woman without prospects at a ball in G...,333,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,12973,4.32,mistress suffragette,A young woman without prospects at a ball in G...,none
350,Stones (One True Child #5),none,L.C. Conn,0,"Book Club, Novels, Fiction, Drama, Contemporary","Deep within the New Zealand bush, lies a plate...",327,2019,English,https://i.gr-assets.com/images/S/compressed.ph...,569,4.31,stones one true child 5,"Deep within the New Zealand bush, lies a plate...",none
355,Mission: Subhero,none,Linda Armstrong,0,"Contemporary, Novels, Fiction, Drama, Book Clu...","With just a few lines of code, one average man...",225,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,2417,4.35,mission subhero,"With just a few lines of code, one average man...",none
356,A Summer of Good-Byes (Blue Triangle Press Boo...,none,Fred Misurella,0,"Fiction, Novels, Adult, Contemporary, Adventur...",PASSION IN PROVENCE:Ben and Lee Alto follow Va...,0,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,1713,4.36,summer good byes blue triangle press book 2,PASSION IN PROVENCE:Ben and Lee Alto follow Va...,none
357,The Scopas Factor,none,Vincent Panettiere,9781543947717,"Fiction, Contemporary, Novels, Drama, Book Clu...","A Hmong ""story cloth,"" a Revolutionary War bat...",326,2018,English,https://i.gr-assets.com/images/S/compressed.ph...,1080,4.31,scopas factor,"A Hmong ""story cloth,"" a Revolutionary War bat...",none


#### English books 
Concat the 2 dataframes with English books and tokenize.

In [14]:
EnglishBooks1=pd.DataFrame(DataForPreprocessing[DataForPreprocessing['language']=='English'])

In [15]:
frames=[NoneLangBooks,EnglishBooks1]
EnglishBooks = pd.concat(frames,axis=0)
EnglishBooks['language'].value_counts()

English    8140
Name: language, dtype: int64

In [16]:
EnglishBooks

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
283,Mistress Suffragette,none,Diana Forbes,1946409073,"Fiction, Novels, Drama, Contemporary, Historic...",A young woman without prospects at a ball in G...,333,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,12973,4.32,mistress suffragette,A young woman without prospects at a ball in G...,none
350,Stones (One True Child #5),none,L.C. Conn,0,"Book Club, Novels, Fiction, Drama, Contemporary","Deep within the New Zealand bush, lies a plate...",327,2019,English,https://i.gr-assets.com/images/S/compressed.ph...,569,4.31,stones one true child 5,"Deep within the New Zealand bush, lies a plate...",none
355,Mission: Subhero,none,Linda Armstrong,0,"Contemporary, Novels, Fiction, Drama, Book Clu...","With just a few lines of code, one average man...",225,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,2417,4.35,mission subhero,"With just a few lines of code, one average man...",none
356,A Summer of Good-Byes (Blue Triangle Press Boo...,none,Fred Misurella,0,"Fiction, Novels, Adult, Contemporary, Adventur...",PASSION IN PROVENCE:Ben and Lee Alto follow Va...,0,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,1713,4.36,summer good byes blue triangle press book 2,PASSION IN PROVENCE:Ben and Lee Alto follow Va...,none
357,The Scopas Factor,none,Vincent Panettiere,9781543947717,"Fiction, Contemporary, Novels, Drama, Book Clu...","A Hmong ""story cloth,"" a Revolutionary War bat...",326,2018,English,https://i.gr-assets.com/images/S/compressed.ph...,1080,4.31,scopas factor,"A Hmong ""story cloth,"" a Revolutionary War bat...",none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8426,The Stove-Junker,none,S.K. Kalsi,9780990779063,none,"In the winter of 2012, 79-year-old Somerset Ga...",338,2015,English,https://i.gr-assets.com/images/S/compressed.ph...,23,3.65,The Stove-Junker,"In the winter of 2012, 79-year-old Somerset Ga...",none
8427,The Treemakers,The Treemakers Trilogy #1,Christina L. Rozelle,1508840466,"Science Fiction, Dystopia, Fantasy, Young Adul...",This is a previously-published alternate cover...,288,2014,English,https://i.gr-assets.com/images/S/compressed.ph...,535,4.23,The Treemakers,This is a previously-published alternate cover...,The Treemakers Trilogy #1
8428,100 Years: A Journey to End a Vicious Cycle,none,Mark L. Baynard,9780986138003,none,"100 Years, A Journey to End a Vicious Cycle is...",290,2015,English,https://i.gr-assets.com/images/S/compressed.ph...,13,4.00,100 Years: A Journey to End a Vicious Cycle,"100 Years, A Journey to End a Vicious Cycle is...",none
8429,Thrive: 30 Inspirational Rags-to-Riches Stories,none,Jason Navallo,1511503882,"Nonfiction, Self Help, Inspirational, Business...",Thrive: 30 Inspirational Rags-to-Riches Storie...,100,2015,English,https://i.gr-assets.com/images/S/compressed.ph...,335,3.89,Thrive: 30 Inspirational Rags-to-Riches Stories,Thrive: 30 Inspirational Rags-to-Riches Storie...,none


In [17]:
#Return lower cases
funct.return_Serieslowercase(EnglishBooks,'title_tokenized')

In [18]:
#Remove stop words
EnglishBooks['title_tokenized'] = EnglishBooks.title_tokenized.apply(func=funct.remove_stopwords)

In [19]:
#Remove non-Ascii Characters
EnglishBooks['title_tokenized'] = EnglishBooks['title_tokenized'].apply(func=funct.remove_nonAscii)

In [20]:
#Remove punctuation
EnglishBooks['title_tokenized'] = EnglishBooks['title_tokenized'].apply(func=funct.remove_punctuation)

In [21]:
EnglishBooks[EnglishBooks['title_tokenized']=='']

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
1765,Why Not Me?,none,Mindy Kaling,9780804138161,"Nonfiction, Autobiography, Memoir, Humor, Audi...",From the author of the belovedNew York Timesbe...,240,2015,English,https://i.gr-assets.com/images/S/compressed.ph...,197538,3.89,,From the author of the belovedNew York Timesbe...,none
285,It,none,Stephen King,9780450411434,"Horror, Fiction, Thriller, Fantasy, Mystery, A...","Welcome to Derry, Maine ...It’s a small city, ...",1116,1986,English,https://i.gr-assets.com/images/S/compressed.ph...,904090,4.24,,"Welcome to Derry, Maine ...It’s a small city, ...",none
905,Out,none,"Natsuo Kirino, Stephen Snyder",9781400078370,"Fiction, Cultural, Japan, Thriller, Mystery, A...",Natsuo Kirino's novel tells a story of random ...,400,1997,English,https://i.gr-assets.com/images/S/compressed.ph...,32844,3.95,,Natsuo Kirino's novel tells a story of random ...,none
1190,Before We Were Yours,none,Lisa Wingate,9780425284681,"Historical, Historical Fiction, Fiction, Audio...","Memphis, 1939.Twelve-year-old Rill Foss and he...",342,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,462425,4.37,,"Memphis, 1939.Twelve-year-old Rill Foss and he...",none
1243,After You,Me Before You #2,Jojo Moyes,9780143108863,"Romance, Fiction, Contemporary, Womens Fiction...",From theNew York Timesbestselling author ofThe...,352,2015,English,https://i.gr-assets.com/images/S/compressed.ph...,330908,3.73,,From theNew York Timesbestselling author ofThe...,Me Before You #2
1322,Me Before You,Me Before You #1,Jojo Moyes,9780143124542,"Romance, Fiction, Contemporary, Womens Fiction...",From the #1New York Timesbestselling author of...,369,2012,English,https://i.gr-assets.com/images/S/compressed.ph...,1311347,4.25,,From the #1New York Timesbestselling author of...,Me Before You #1
2285,More Than This,none,Patrick Ness,9781406350487,"Young Adult, Science Fiction, LGBT, Fiction, D...","A boy drowns, desperate and alone in his final...",480,2013,English,https://i.gr-assets.com/images/S/compressed.ph...,59861,3.95,,"A boy drowns, desperate and alone in his final...",none
2979,جاناتان، مرغ دریایی,none,"Richard Bach, فاطمه محمدی",9789641750338,"Fiction, Classics, Philosophy, Fantasy, Spirit...",NaN,110,1970,English,https://i.gr-assets.com/images/S/compressed.ph...,221751,3.85,,NaN,none
3391,It Had to Be You,Chicago Stars #1,Susan Elizabeth Phillips,9780380776832,"Romance, Contemporary Romance, Contemporary, S...",What if a woman who knows nothing about sports...,376,1994,English,https://i.gr-assets.com/images/S/compressed.ph...,62779,4.02,,What if a woman who knows nothing about sports...,Chicago Stars #1
4227,We,none,"Yevgeny Zamyatin, Clarence Brown",9780140185850,"Fiction, Science Fiction, Dystopia, Classics, ...",The exhilarating dystopian novel that inspired...,226,1924,English,https://i.gr-assets.com/images/S/compressed.ph...,82948,3.90,,The exhilarating dystopian novel that inspired...,none


**Caution:** The result shows books where the title only contained stopwords, e.g. Stephan King "It". There the title is now emtpy. I need to fill up the ones with empty title as they cannot be vectorized later if they are emtpy.

In [22]:
rows =[6497,5836,4902,4567,4345,4227,3391,2285,1322,1243, 1190,905,285,1765]
for i in rows:
    EnglishBooks['title_tokenized'][i]=EnglishBooks['title'][i]
EnglishBooks[EnglishBooks['title_tokenized']=='']

<ipython-input-22-cb8a8a454392>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EnglishBooks['title_tokenized'][i]=EnglishBooks['title'][i]


,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
2979,جاناتان، مرغ دریایی,none,"Richard Bach, فاطمه محمدی",9789641750338,"Fiction, Classics, Philosophy, Fantasy, Spirit...",NaN,110,1970,English,https://i.gr-assets.com/images/S/compressed.ph...,221751,3.85,,NaN,none


In [23]:
#check if successful 
EnglishBooks.loc[5836,:]

title                                                                   You
series                                                               You #1
authors                                                     Caroline Kepnes
isbn                                                          9781501195433
genres                    Thriller, Fiction, Mystery, Mystery Thriller, ...
description               When a beautiful aspiring writer strides into ...
pages                                                                   422
year                                                                   2014
language                                                            English
cover_image               https://i.gr-assets.com/images/S/compressed.ph...
total_number_ratings                                                 184107
average_rating                                                         3.92
title_tokenized                                                         You
desccription

#### Remove the last Arabic book at index 2979

In [24]:
EnglishBooks = EnglishBooks.drop(2979)

#### Show final dataframe after title tokenizing

In [25]:
EnglishBooks.shape

(8139, 15)

**After selecting only the English books, there are 8139 of 8431 books left in the database.**

### Preprocess Book Descriptions

In [26]:
EnglishBooksDescr=EnglishBooks.copy()
EnglishBooksDescr.head()

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
283,Mistress Suffragette,none,Diana Forbes,1946409073,"Fiction, Novels, Drama, Contemporary, Historic...",A young woman without prospects at a ball in G...,333,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,12973,4.32,mistress suffragette,A young woman without prospects at a ball in G...,none
350,Stones (One True Child #5),none,L.C. Conn,0,"Book Club, Novels, Fiction, Drama, Contemporary","Deep within the New Zealand bush, lies a plate...",327,2019,English,https://i.gr-assets.com/images/S/compressed.ph...,569,4.31,stones one true child 5,"Deep within the New Zealand bush, lies a plate...",none
355,Mission: Subhero,none,Linda Armstrong,0,"Contemporary, Novels, Fiction, Drama, Book Clu...","With just a few lines of code, one average man...",225,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,2417,4.35,mission subhero,"With just a few lines of code, one average man...",none
356,A Summer of Good-Byes (Blue Triangle Press Boo...,none,Fred Misurella,0,"Fiction, Novels, Adult, Contemporary, Adventur...",PASSION IN PROVENCE:Ben and Lee Alto follow Va...,0,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,1713,4.36,summer good byes blue triangle press book 2,PASSION IN PROVENCE:Ben and Lee Alto follow Va...,none
357,The Scopas Factor,none,Vincent Panettiere,9781543947717,"Fiction, Contemporary, Novels, Drama, Book Clu...","A Hmong ""story cloth,"" a Revolutionary War bat...",326,2018,English,https://i.gr-assets.com/images/S/compressed.ph...,1080,4.31,scopas factor,"A Hmong ""story cloth,"" a Revolutionary War bat...",none


In [27]:
#Return lower cases
funct.return_Serieslowercase(EnglishBooksDescr,'desccription_tokenized')

In [28]:
#Remove stop words
EnglishBooksDescr['desccription_tokenized']= EnglishBooksDescr.desccription_tokenized.apply(func=funct.remove_stopwords)

In [29]:
#Remove non-Ascii Characters
EnglishBooksDescr['desccription_tokenized'] = EnglishBooksDescr['desccription_tokenized'].apply(func=funct.remove_nonAscii)

In [30]:
#Remove punctuation
EnglishBooksDescr['desccription_tokenized'] = EnglishBooksDescr['desccription_tokenized'].apply(func=funct.remove_punctuation)

In [31]:
EnglishBooksDescr['desccription_tokenized']

283     young woman without prospects ball gilded age ...
350     deep within new zealand bush lies plateau clea...
355     lines code one average man change world and se...
356     passion provence ben lee alto follow van goghs...
357     hmong story cloth revolutionary war battle fla...
                              ...                        
8426    winter 2012 79 year old somerset garden travel...
8427    previously published alternate cover edition a...
8428    100 years journey end vicious cycle true story...
8429    thrive 30 inspirational rags to riches stories...
8430    new york timesbestsellerjeff shaara returns ci...
Name: desccription_tokenized, Length: 8139, dtype: object

#### Check books with no descriptions and insert ''

In [32]:
EnglishBooksDescr[EnglishBooksDescr['desccription_tokenized']=='nan']

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
752,Last Chance to See,none,"Douglas Adams, Mark Carwardine",9780345371980,"Nonfiction, Science, Travel, Environment, Natu...",NaN,256,1990,English,https://i.gr-assets.com/images/S/compressed.ph...,24202,4.26,last chance see,nan,none
1433,The Loop,none,Nicholas Evans,9780552144957,"Fiction, Romance, Animals, Drama, Contemporary...",NaN,511,1998,English,https://i.gr-assets.com/images/S/compressed.ph...,11567,3.92,loop,nan,none
1501,Holy Bible: The New King James Version,none,Anonymous,9780718015596,"Religion, Christian, Nonfiction, Classics, Ref...",NaN,1536,1536,English,https://i.gr-assets.com/images/S/compressed.ph...,7153,4.37,holy bible new king james version,nan,none
1847,Book of Leo: Prequel to Mystical Trash,none,Mary Ramsey,0,"Epic, Novels, Suspense, Thriller, Drama, Conte...",NaN,83,2019,English,https://i.gr-assets.com/images/S/compressed.ph...,568,4.32,book leo prequel mystical trash,nan,none
2187,A Simple Plan,none,Scott Smith,9780307279958,"Fiction, Thriller, Mystery, Crime, Suspense, D...",NaN,432,1993,English,https://i.gr-assets.com/images/S/compressed.ph...,34654,3.89,simple plan,nan,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8361,Pictures of Hollis Woods,none,Patricia Reilly Giff,9780440415787,"Young Adult, Realistic Fiction, Fiction, Child...",NaN,166,2002,English,https://i.gr-assets.com/images/S/compressed.ph...,28196,4.02,pictures hollis woods,nan,none
8369,The Tree Of Man,none,Patrick White,9780099324515,"Fiction, Cultural, Australia, Novels, Classics...",NaN,480,1955,English,https://i.gr-assets.com/images/S/compressed.ph...,1762,3.95,tree man,nan,none
8395,The Last of Her Kind,none,JoAnn Rackear Goldrich,9781505308266,none,NaN,240,2015,English,https://i.gr-assets.com/images/S/compressed.ph...,22,3.68,last kind,nan,none
8398,South of Reality,none,A.I. Alexen,9781301529,none,NaN,0,2012,English,https://i.gr-assets.com/images/S/compressed.ph...,6,4.50,south reality,nan,none


In [33]:
funct.replace_in_column(EnglishBooksDescr, 'desccription_tokenized', 'nan', '')

In [34]:
funct.replace_in_column(EnglishBooksDescr, 'desccription_tokenized', 'none', '')

In [35]:
EnglishBooksDescr[EnglishBooksDescr['desccription_tokenized']=='']

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
752,Last Chance to See,none,"Douglas Adams, Mark Carwardine",9780345371980,"Nonfiction, Science, Travel, Environment, Natu...",NaN,256,1990,English,https://i.gr-assets.com/images/S/compressed.ph...,24202,4.26,last chance see,,none
935,Understanding China: Learning from China's Pas...,none,Stefan Piech,0,"Cultural, China, Politics, Nonfiction, History...",None,0,2020,English,https://i.gr-assets.com/images/S/compressed.ph...,32,4.13,understanding china learning china past presen...,,none
983,Lift: Five Practices Great Managers Do Consist...,none,Raymond Wheeler,0,"How To, Nonfiction, Education, Inspirational, ...",None,0,2020,English,https://i.gr-assets.com/images/S/compressed.ph...,437,4.38,lift five practices great managers consistentl...,,none
1129,The Spirit of Prayer: The Believer's Authority...,none,Tolulope Oyewole,1527285502,"Contemporary, Novels, Drama",None,0,2021,English,https://i.gr-assets.com/images/S/compressed.ph...,553,4.30,spirit prayer believer authority earth sons go...,,none
1163,PACKFIRE: Simon Pack # 9,none,John M Vermillion,0,"Contemporary, Fiction, Drama, Novels",None,0,2020,English,https://i.gr-assets.com/images/S/compressed.ph...,1126,4.31,packfire simon pack 9,,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8361,Pictures of Hollis Woods,none,Patricia Reilly Giff,9780440415787,"Young Adult, Realistic Fiction, Fiction, Child...",NaN,166,2002,English,https://i.gr-assets.com/images/S/compressed.ph...,28196,4.02,pictures hollis woods,,none
8369,The Tree Of Man,none,Patrick White,9780099324515,"Fiction, Cultural, Australia, Novels, Classics...",NaN,480,1955,English,https://i.gr-assets.com/images/S/compressed.ph...,1762,3.95,tree man,,none
8395,The Last of Her Kind,none,JoAnn Rackear Goldrich,9781505308266,none,NaN,240,2015,English,https://i.gr-assets.com/images/S/compressed.ph...,22,3.68,last kind,,none
8398,South of Reality,none,A.I. Alexen,9781301529,none,NaN,0,2012,English,https://i.gr-assets.com/images/S/compressed.ph...,6,4.50,south reality,,none


### Preprocess Column Series

In [36]:
EnglishBooksSeries= EnglishBooksDescr.copy()

In [37]:
#Return lower cases
funct.return_Serieslowercase(EnglishBooksSeries,'series_tokenized')

In [38]:
#Remove stop words
EnglishBooksSeries['series_tokenized']= EnglishBooksSeries.series_tokenized.apply(func=funct.remove_stopwords)

In [39]:
#Remove non-Ascii Characters
EnglishBooksSeries['series_tokenized'] = EnglishBooksSeries['series_tokenized'].apply(func=funct.remove_nonAscii)

In [40]:
#Remove punctuation
EnglishBooksSeries['series_tokenized'] = EnglishBooksSeries['series_tokenized'].apply(func=funct.remove_punctuation)

In [41]:
EnglishBooksSeries['series_tokenized']

283                                      none
350                                      none
355                                      none
356                                      none
357                                      none
                        ...                  
8426                                     none
8427                     treemakers trilogy 1
8428                                     none
8429                                     none
8430    civil war 1861 1865 western theater 3
Name: series_tokenized, Length: 8139, dtype: object

#### Check all books with no series and enter ''

In [42]:
EnglishBooksSeries[EnglishBooksSeries['series_tokenized']=='none']

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
283,Mistress Suffragette,none,Diana Forbes,1946409073,"Fiction, Novels, Drama, Contemporary, Historic...",A young woman without prospects at a ball in G...,333,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,12973,4.32,mistress suffragette,young woman without prospects ball gilded age ...,none
350,Stones (One True Child #5),none,L.C. Conn,0,"Book Club, Novels, Fiction, Drama, Contemporary","Deep within the New Zealand bush, lies a plate...",327,2019,English,https://i.gr-assets.com/images/S/compressed.ph...,569,4.31,stones one true child 5,deep within new zealand bush lies plateau clea...,none
355,Mission: Subhero,none,Linda Armstrong,0,"Contemporary, Novels, Fiction, Drama, Book Clu...","With just a few lines of code, one average man...",225,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,2417,4.35,mission subhero,lines code one average man change world and se...,none
356,A Summer of Good-Byes (Blue Triangle Press Boo...,none,Fred Misurella,0,"Fiction, Novels, Adult, Contemporary, Adventur...",PASSION IN PROVENCE:Ben and Lee Alto follow Va...,0,2017,English,https://i.gr-assets.com/images/S/compressed.ph...,1713,4.36,summer good byes blue triangle press book 2,passion provence ben lee alto follow van goghs...,none
357,The Scopas Factor,none,Vincent Panettiere,9781543947717,"Fiction, Contemporary, Novels, Drama, Book Clu...","A Hmong ""story cloth,"" a Revolutionary War bat...",326,2018,English,https://i.gr-assets.com/images/S/compressed.ph...,1080,4.31,scopas factor,hmong story cloth revolutionary war battle fla...,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8423,The Testament of Vida Tremayne,none,Sarah Vincent,9781910153109,"Fiction, Novels","A lonely novelist, A devoted fan, A journal th...",400,2014,English,https://i.gr-assets.com/images/S/compressed.ph...,93,3.90,testament vida tremayne,lonely novelist devoted fan journal speaks uns...,none
8425,Roommate of Fire,none,Aiden Bates,0,"Romance, M M Romance","A Gay Romance Erotica MMNeil, a man desperate ...",32,2015,English,https://i.gr-assets.com/images/S/compressed.ph...,43,3.37,roommate fire,gay romance erotica mmneil man desperate escap...,none
8426,The Stove-Junker,none,S.K. Kalsi,9780990779063,none,"In the winter of 2012, 79-year-old Somerset Ga...",338,2015,English,https://i.gr-assets.com/images/S/compressed.ph...,23,3.65,stove junker,winter 2012 79 year old somerset garden travel...,none
8428,100 Years: A Journey to End a Vicious Cycle,none,Mark L. Baynard,9780986138003,none,"100 Years, A Journey to End a Vicious Cycle is...",290,2015,English,https://i.gr-assets.com/images/S/compressed.ph...,13,4.00,100 years journey end vicious cycle,100 years journey end vicious cycle true story...,none


In [43]:
EnglishBooksSeries[EnglishBooksSeries['series_tokenized']=='']

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
5233,Stories Of The Prophets,البداية والنهاية,"ابن كثير, Ibn Kathir, Rashad Ahmad Azami",9789960892269,"Religion, Islam, History, Nonfiction, Biograph...",The Stories of the Prophets have been compiled...,576,1981,English,https://i.gr-assets.com/images/S/compressed.ph...,5749,4.49,stories prophets,stories prophets compiled al bidayah wan nihay...,
6794,The End of the World,نهاية العالم,محمد عبدالرحمن العريفي,0,"Religion, Islam, Nonfiction",In recent times things have become very confus...,410,2010,English,https://i.gr-assets.com/images/S/compressed.ph...,4105,4.13,end world,recent times things become confusing begun see...,


#### Delete another 2 found arabic books at index 5233 and 6794

In [44]:
in_drop=[5233,6794]
EnglishBooksSeries = EnglishBooksSeries.drop(in_drop)
EnglishBooksSeries.shape

(8137, 15)

**After removing these two books, there are 8137 books left in the dataset.**

#### Replace all books with no series with ''

In [45]:
funct.replace_in_column(EnglishBooksSeries, 'series_tokenized', 'none', '')
EnglishBooksSeries['series_tokenized'].value_counts()

                               5398
1                                10
chronicles narnia                 6
james bond                        5
foundation                        5
                               ... 
villains 1                        1
graceling realm 2                 1
criminal profiler 3               1
selection 1                       1
celtic cousins adventures 2       1
Name: series_tokenized, Length: 2616, dtype: int64

In [46]:
EnglishBooksSeries[EnglishBooksSeries['series'].str.match('Harry Potter')]

,title,series,authors,isbn,genres,description,pages,year,language,cover_image,total_number_ratings,average_rating,title_tokenized,desccription_tokenized,series_tokenized
1,Harry Potter and the Sorcerer's Stone,Harry Potter #1,J.K. Rowling,0,"Fantasy, Fiction, Young Adult, Magic, Children...",Harry Potter's life is miserable. His parents ...,309,1997,English,https://i.gr-assets.com/images/S/compressed.ph...,8003482,4.47,harry potter sorcerer s stone,harry potter s life miserable parents dead he ...,harry potter 1
13,Harry Potter and the Deathly Hallows,Harry Potter #7,J.K. Rowling,747591059,"Fantasy, Young Adult, Fiction, Magic, Children...","It's no longer safe for Harry at Hogwarts, so ...",759,2007,English,https://i.gr-assets.com/images/S/compressed.ph...,3111326,4.61,harry potter deathly hallows,longer safe harry hogwarts best friends ron he...,harry potter 7
25,Harry Potter and the Prisoner of Azkaban,Harry Potter #3,"J.K. Rowling, Mary GrandPré",9780439655484,"Fantasy, Young Adult, Fiction, Magic, Children...","For twelve long years, the dread fortress of A...",435,1999,English,https://i.gr-assets.com/images/S/compressed.ph...,3223918,4.57,harry potter prisoner azkaban,twelve long years dread fortress azkaban held ...,harry potter 3
30,Harry Potter and the Goblet of Fire,Harry Potter #4,"J.K. Rowling, Mary GrandPré",0,"Fantasy, Young Adult, Fiction, Magic, Children...",Harry Potter is midway through his training as...,734,2000,English,https://i.gr-assets.com/images/S/compressed.ph...,2907124,4.56,harry potter goblet fire,harry potter midway training wizard coming age...,harry potter 4
33,Harry Potter and the Half-Blood Prince,Harry Potter #6,J.K. Rowling,747581088,"Fantasy, Young Adult, Fiction, Magic, Children...",The war against Voldemort is not going well; e...,652,2005,English,https://i.gr-assets.com/images/S/compressed.ph...,2714400,4.57,harry potter half blood prince,war voldemort going well even muggle governmen...,harry potter 6
73,Harry Potter and the Chamber of Secrets,Harry Potter #2,"J.K. Rowling, Mary GrandPré, ویدا اسلامیه",1408855666,"Fantasy, Young Adult, Fiction, Magic, Children...",Ever since Harry Potter had come home for the ...,341,1998,English,https://i.gr-assets.com/images/S/compressed.ph...,3089063,4.43,harry potter chamber secrets,ever since harry potter come home summer dursl...,harry potter 2
279,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré",747551006,"Fantasy, Young Adult, Fiction, Magic, Children...",There is a door at the end of a silent corrido...,870,2003,English,https://i.gr-assets.com/images/S/compressed.ph...,2800355,4.50,harry potter order phoenix,door end silent corridor its haunting harry po...,harry potter 5


## Save to Csv file

In [47]:
EnglishBooksSeries.to_csv('DataFinal\DataPreprocessed.csv')